In [1]:
# First import
import os
import sys

# project_dir = os.path.realpath(os.path.join(os.getcwd(), '..','..','..'))
# tool_dir = os.path.join(project_dir, 'Utils')
# sys.path.append(tool_dir)
print("First import is completed.")

First import is completed.


In [2]:
# Second import
from workers import *
import ta
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('seaborn')

print("Second import is completed.")

Second import is completed.


In [3]:
class technicalIndicator:
    def __init__(self, ticker, start_date, end_date):

        self.ticker = ticker
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.df = get_all_price(self.ticker, self.start_date, self.end_date)

        self.columns = {'open','high','low','close','volume'}
        assert self.columns.issubset(self.df.columns)
        
        self.df = self.df
        self.open = self.df.open
        self.high = self.df.high
        self.low = self.df.low
        self.close = self.df.close
        self.volume = self.df.volume
        self.atr()
        self.rsi()
        
    def run(self, strategy='reversal'):
        if strategy == 'reversal':
            index = self.reversal_functions()
            return self.df[index]
    
    def reversal_functions(self):
        return self.close_minus_open_filter() & \
               self.open_minus_low_filter() & \
               self.volume_filter() & \
               self.neg_cumret_filter() & \
               self.rsi_filter()
    
    #other functionality
    def persistence(self, window=30):
        rolling_high = self.high.rolling(window).max()
        rolling_low = self.low.rolling(window).min()
        
        high_persistence = pd.DataFrame(rolling_high.dropna().drop_duplicates(keep='first'))
        high_persistence['start_date'] = high_persistence.index
        high_persistence['end_date'] = high_persistence['start_date'].shift(-1)
        high_persistence.loc[high_persistence.index[-1], 'end_date'] = self.end_date
        high_persistence['duration'] = high_persistence['end_date'] - high_persistence['start_date']
        sorted_high_persistence = high_persistence.sort_values('duration', ascending=False)
        
        low_persistence = pd.DataFrame(rolling_low.dropna().drop_duplicates(keep='first'))
        low_persistence['start_date'] = low_persistence.index
        low_persistence['end_date'] = low_persistence['start_date'].shift(-1)
        low_persistence.loc[low_persistence.index[-1], 'end_date'] = self.end_date
        low_persistence['duration'] = low_persistence['end_date'] - low_persistence['start_date']
        sorted_low_persistence = low_persistence.sort_values('duration', ascending=False)
        
        return sorted_high_persistence, sorted_low_persistence
        
    #filters
    def close_minus_open_filter(self, k=0.5, strength = 'strong'):
        """
        close - open >= K*atr
        strength = 'strong', 'weak'
        """
        if strength == 'weak':
            index = abs(self.close - self.open) <= k*self.atr
        else:
            index = self.close - self.open >= k*self.atr
        return index
    
    def open_minus_low_filter(self, k=0.5):
        """
        Open - Low >= K*(Close - Open)
        """
        index = self.open - self.low >= k*(self.close-self.open)
        return index

    def volume_filter(self, periods=5):
        """
        Current volume > volume MA 
        """
        lag_volume = self.volume.rolling(periods).mean().shift()
        index = self.volume > lag_volume
        return index

    def neg_cumret_filter(self, periods=5):
        """
        Cumulative return is negative for previous n days
        """
        lag_close = self.close.shift(periods)
        index = self.close < lag_close
        return index
    
    def rsi_filter(self, periods=5, threshold = None):
        """
        Current RSI < min(RSI in the past n days)
        """
        if threshold is None:
            rolling_rsi = self.rsi.rolling(periods).min().shift()
            index = self.rsi < rolling_rsi
        else:
            index = self.rsi <= threshold
        return index
        
    #technical indicators
    def atr(self, n=14):
        data = self.df.copy()
        data['tr0'] = abs(self.high - self.low)
        data['tr1'] = abs(self.high - self.close.shift())
        data['tr2'] = abs(self.low - self.close.shift())
        tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
        self.atr = self.wwma(tr, n)
    
    def wwma(self, values, n):
        """
        J. Welles Wilder's EMA 
        """
        return values.ewm(alpha=1/n, adjust=False).mean()
    
    def rsi(self):
        self.rsi = ta.momentum.rsi(self.close, window=14)
    
    @classmethod
    def plot_filtered_date(cls, full_df, filtered_df, persistence_df=None):
        assert 'close' in full_df.columns, 'close column missing'
        close = full_df.close
        filtered_date = filtered_df.index
        
        plt.figure(figsize=(10,4))
        plt.plot(close, label='close')
        plt.scatter(filtered_date, close.loc[filtered_date], c='r')
        plt.legend()
        if persistence_df is not None:
            for index, row in persistence_df.iterrows():
                start_date = row['start_date']
                end_date = row['end_date']
                plt.fill_between(close.loc[start_date: end_date].index, close.min(),
                                 close.loc[start_date: end_date].to_numpy(),
                                 color='g', alpha=0.3)

In [6]:
%%time 
ticker_list = pd.read_csv('DLW_stock_all.csv')['Symbol'].to_list()
start_date = '2021-07-01'
end_date = '2021-08-03'

result_dict = {}
n = 1
for ticker in ticker_list:
    try:
        TI = technicalIndicator(ticker, start_date, end_date)
        TI.run(strategy='reversal')

        f1 = TI.close_minus_open_filter(0.2, strength = 'weak')
        f2 = TI.open_minus_low_filter(0.1)
        f3 = TI.volume_filter(5)
        f4 = TI.neg_cumret_filter()
        f5 = TI.rsi_filter(periods=5, threshold = 40)
        filtered = TI.df[f1&f2&f3&f4&f5]

        if filtered.empty:
            print("{}: {} is empty".format(n, ticker))
        else:
            result_dict.update({ticker: filtered})
            print("{}: {} is done!".format(n, ticker))
        # persistence_high, persistence_low = TI.persistence()
        n += 1
    except:
        print("Error! {} skipped.".format(ticker))
        n += 1

[*********************100%***********************]  1 of 1 completed
1: TSM is empty
[*********************100%***********************]  1 of 1 completed
2: MVIS is empty
[*********************100%***********************]  1 of 1 completed
3: KODK is done!
[*********************100%***********************]  1 of 1 completed
4: MU is empty
[*********************100%***********************]  1 of 1 completed
5: CDR is done!
[*********************100%***********************]  1 of 1 completed
6: ADBE is empty
[*********************100%***********************]  1 of 1 completed
7: TSLA is empty
[*********************100%***********************]  1 of 1 completed
8: IRDM is empty
[*********************100%***********************]  1 of 1 completed
9: DAL is empty
[*********************100%***********************]  1 of 1 completed
10: WFC is empty
[*********************100%***********************]  1 of 1 completed
11: LAC is empty
[*********************100%***********************]  1 of 1 

Exception in thread Thread-503:
Traceback (most recent call last):
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\site-packages\yfinance\multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\site-packages\yfinance\multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "C:\Users\liwei\AppData\Local\Programs\Python\Python39\lib\site-packages\yfinance\bas

Error! BRK/B skipped.
[*********************100%***********************]  1 of 1 completed
151: SO is empty
[*********************100%***********************]  1 of 1 completed
152: CRM is empty
[*********************100%***********************]  1 of 1 completed
153: VALE is empty
[*********************100%***********************]  1 of 1 completed
154: GILD is empty
[*********************100%***********************]  1 of 1 completed
155: LUV is empty
[*********************100%***********************]  1 of 1 completed
156: RUN is empty
[*********************100%***********************]  1 of 1 completed
157: UBER is done!
[*********************100%***********************]  1 of 1 completed
158: BABA is done!
[*********************100%***********************]  1 of 1 completed
159: CSCO is empty
[*********************100%***********************]  1 of 1 completed
160: DLR is empty
[*********************100%***********************]  1 of 1 completed
161: HON is empty
[****************

In [7]:
result_dict.keys()

dict_keys(['KODK', 'CDR', 'NKLA', 'FDX', 'BILI', 'NIO', 'RH', 'SWN', 'HUYA', 'FSLY', 'KC', 'BIDU', 'JMIA', 'CNK', 'KWEB', 'MGM', 'IQ', 'TDOC', 'TME', 'CCI', 'ADM', 'UBER', 'BABA', 'PDD', 'PCG', 'VIRT', 'GRPN', 'FSLR', 'ATVI', 'KHC', 'WIMI', 'DOW', 'CHAU', 'SPOT', 'DD', 'W', 'VLDR', 'WBA', 'XOM', 'IMO', 'GOTU', 'PINS', 'EH', 'DM', 'DOYU', 'DADA', 'NIU', 'RIDE', 'INTC', 'WLTW', 'WDC', 'DUO', 'BEKE', 'XPEV', 'LU', 'AI', 'RAAS', 'MOGU'])

In [44]:
append_list = []
for key in result_dict.keys():
    append_list.append(result_dict[key].head(1))

In [45]:
pd.concat(append_list, keys = result_dict.keys())

,,open,high,low,close,adj close,volume
,Date,,,,,,
KODK,2021-07-27,7.190000,7.240000,6.970000,7.180000,7.180000,1351100
CDR,2021-07-28,15.800000,16.230000,15.130000,15.790000,15.790000,47300
NKLA,2021-07-30,11.900000,12.570000,11.790000,11.870000,11.870000,16533600
FDX,2021-07-30,280.000000,282.000000,277.790009,279.950012,279.950012,2335500
BILI,2021-07-26,88.050003,91.315002,85.709999,87.459999,87.459999,12304200
NIO,2021-07-26,42.669998,44.880001,41.930000,43.169998,43.169998,54759200
RH,2021-07-30,663.580017,675.000000,658.830017,664.080017,664.080017,290900
SWN,2021-07-30,4.670000,4.810000,4.560000,4.710000,4.710000,16920500
HUYA,2021-07-26,13.020000,13.800000,12.750000,13.100000,13.100000,4720300


In [ ]:
technicalIndicator.plot_filtered_date(TI.df, filtered, persistence_high[:5])
plt.title(ticker + ' reversal pattern')